In [1]:
!pip install lime

In [2]:
import tensorflow as tf
import os
import numpy as np
import random
import pickle
import pandas as pd

import matplotlib.pyplot as plt

import lime
from lime.lime_text import LimeTextExplainer

from keras.preprocessing.sequence import pad_sequences

from torch.utils.data import Dataset
import torch

from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve

import gc
import re

def set_seeds(seed=123):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
set_seeds(seed=123)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# class used to store datasets in this project (required for object loading from pickle)
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, evidences=None):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.evidences=evidences

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
#             token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])   

In [4]:
# Global variables
task = "eraser_movie"
explain_count = 40
skip = 0
models_dataset = "models-test2"


# Loading the test dataset
with open(f"/kaggle/input/balanced-datasets/{task}_test.obj", 'rb') as pickle_file:
    test_dataset = pickle.load(pickle_file)


In [5]:
# Method used to get lime values of n instances from the test dataset staarting from the i-th instance using given model and tokenizer
def get_xai(i, n, model=None, tokenizer=None):

    # Helper method used to get probas for all tested model types
    def prob(data):
        if str(type(model))=="<class 'keras.src.models.sequential.Sequential'>":
            # Preprocess the data
            max_len = model.input_shape[1] if hasattr(model, 'input_shape') else 512  
            sequences = tokenizer.texts_to_sequences(data)
            padded_sequences = pad_sequences(sequences, maxlen=max_len)
            
            # Return probas
            probabilities = model.predict(padded_sequences, verbose=0)
            return np.array([[1-x[0], x[0]] for x in probabilities])
            
          
        elif str(type(model))=="<class 'sklearn.svm._classes.SVC'>":
            # Preprocess the data
            data = tokenizer.transform(data)
            # Return probas
            return np.array(model.predict_proba(data))

        else:
            # Tokenize all texts in the input list
            inputs = tokenizer(data, padding=True, truncation=True, return_tensors="pt")
            
            # Move inputs to the correct device
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            
            # Get model predictions
            with torch.no_grad():
                pred = model(input_ids=input_ids.long(), 
                 attention_mask=attention_mask).logits
            
            # Convert logits to probabilities using softmax
            probs = torch.nn.functional.softmax(pred, dim=-1).cpu().detach().numpy()
            return probs
            
    

    
    attributions = []

    # Load explainer object
    explainer = LimeTextExplainer()
    
    for j in range(i, i+n):
        # Getting initial parameters
        set_seeds(seed=123)
        input_text = test_dataset.inputs[j]
        num_features = len(re.split(r'\W+',test_dataset.inputs[i])) 

        # Calculating the explanations
        explanation = explainer.explain_instance(
            input_text,
            lambda data: prob(data), 
            num_features=num_features if str(type(model))!="<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>" else len(test_dataset.tokenized_inputs[j].tokens),
            num_samples=400 if str(type(model))=="<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>" else 1000
        )

        # Saving the results
        attributions.append(explanation)

        # Memory clean up
        torch.cuda.empty_cache()
        gc.collect()

    return attributions



In [12]:
# Looping through files in search of .csv file with model performance measures
for file in os.listdir(f"/kaggle/input/{models_dataset}/"):
    if ".csv"in file:
        df = pd.read_csv(f"/kaggle/input/{models_dataset}/"+file)

# Creatinng column with model names       
df["filename"] = "eraser_movie_"+df["Imbalance %"].astype(str)+"_"+df["Balancing method"]+"_"+df["Model ID"].astype(str)+"_"+df["Model name"]+".obj"
df["filename"] = df["filename"].apply(lambda x: x if x in os.listdir(f"/kaggle/input/{models_dataset}/") else x.replace("eraser_movie", "erqaser_movie"))

# List of bad models
bad_models = list(df[(df["f1"]==0) | (df["acc"]<0.6)]["filename"])
print(f"This set of models contains {len(bad_models)} bad models, which are {np.round(100*len(bad_models)/len(df),2)}% of all models. XAI will not be calculated for them")

This set of models contains 127 bad models, which are 89.44% of all models. XAI will not be calculated for them


In [7]:
results = []
i = 0

# Looping through the files in dataset
for file in os.listdir(f"/kaggle/input/{models_dataset}/"):

    # Filtering files to get only models (not  tokenizers) that are not bad models (unless this is the original model)
    if "tok" != file[:3] and ".obj" in file and (file not in bad_models or "original" in file):

        # Skipping selected few models (code had to be run multiple times as memory resources were unsufficient to explain all models in one go)
        i+=1
        if i<= skip and "original" not in file: continue

        # Loading the model
        print(file)
        with open(f"/kaggle/input/{models_dataset}/"+file, 'rb') as pickle_file:
            model = pickle.load(pickle_file)

            # Using GPU for DistilBERT and loading its tokenizer
            if file.split("_")[-1].split('.obj')[0]=="DistilBERT":
                model.to(device)
                tokenizer = test_dataset.tokenizer
            # Loaidng tokenizers for other models
            else:
                with open(f"/kaggle/input/{models_dataset}/"+"tok_"+file, 'rb') as tok_file:
                    tokenizer = pickle.load(tok_file) 

            # Calculating explanations
            result = get_xai(0, explain_count, model, tokenizer)
            results.append([file, result])


erqaser_movie_20_LLama_4_LSTM.obj
eraser_movie_50_Contextual_word_embedding_4_LSTM.obj
erqaser_movie_50_LLama_1_LSTM.obj
erqaser_movie_10_LLama_0_LSTM.obj
eraser_movie_50_paraphrase_3_LSTM.obj
erqaser_movie_50_LLama_3_LSTM.obj
erqaser_movie_20_LLama_1_LSTM.obj
eraser_movie_20_Contextual_word_embedding_0_LSTM.obj
erqaser_movie_10_LLama_1_LSTM.obj
erqaser_movie_20_LLama_complex_3_LSTM.obj
erqaser_movie_20_LLama_complex_4_LSTM.obj
erqaser_movie_10_LLama_complex_1_LSTM.obj
eraser_movie_50_Spelling_mistake_2_LSTM.obj
erqaser_movie_10_LLama_complex_3_LSTM.obj
erqaser_movie_50_LLama_4_LSTM.obj
erqaser_movie_50_LLama_complex_3_LSTM.obj
erqaser_movie_50_LLama_0_LSTM.obj
erqaser_movie_10_LLama_complex_2_LSTM.obj
erqaser_movie_10_LLama_3_LSTM.obj
erqaser_movie_10_LLama_complex_4_LSTM.obj
erqaser_movie_50_LLama_complex_0_LSTM.obj
eraser_movie_50_paraphrase_1_LSTM.obj
erqaser_movie_10_LLama_2_LSTM.obj
erqaser_movie_50_LLama_2_LSTM.obj
eraser_movie_10_ROS_4_LSTM.obj
erqaser_movie_10_LLama_4_LSTM.obj

In [8]:
# Method used to properly order LIME values as it was done in order of words
def tokenize_evidence(words, exp):
    explanations = {k:v for k, v in exp.as_list()}
    xai_lime = [explanations[word] if word in explanations.keys() else 0  for word in words]
    return xai_lime


# Clearing up the results to anticipated format
res = []
for j in range(len(results)):
    print(results[j][0])
    for i in range(explain_count):
        
        exp = results[j][1][i]
        words = [word for word in re.split(r'\W+',test_dataset.inputs[i]) if word != ""]
        xai_lime = tokenize_evidence(words, exp)

        # Saving results for further analysis
        explanation = {}
        explanation["model"] = results[j][0]                                               # Model object name
        explanation["id"] = i                                                              # Explained instance id
        explanation["words_sentence"] = words                                              # List of all words in a given instance
        explanation["lime_sentence"] = xai_lime                                            # Lime values for all words in a given instance
        explanation["words_word"] = np.unique(words)                                       # List of unique words in a given instance
        explanation["lime_word"] = tokenize_evidence(explanation["words_word"], exp)       # Lime values for unique words in a given instance
        res.append(explanation)
        
pd.DataFrame(res).to_csv("XAI_results_final_LSTM.csv")

erqaser_movie_20_LLama_4_LSTM.obj
eraser_movie_50_Contextual_word_embedding_4_LSTM.obj
erqaser_movie_50_LLama_1_LSTM.obj
erqaser_movie_10_LLama_0_LSTM.obj
eraser_movie_50_paraphrase_3_LSTM.obj
erqaser_movie_50_LLama_3_LSTM.obj
erqaser_movie_20_LLama_1_LSTM.obj
eraser_movie_20_Contextual_word_embedding_0_LSTM.obj
erqaser_movie_10_LLama_1_LSTM.obj
erqaser_movie_20_LLama_complex_3_LSTM.obj
erqaser_movie_20_LLama_complex_4_LSTM.obj
erqaser_movie_10_LLama_complex_1_LSTM.obj
eraser_movie_50_Spelling_mistake_2_LSTM.obj
erqaser_movie_10_LLama_complex_3_LSTM.obj
erqaser_movie_50_LLama_4_LSTM.obj
erqaser_movie_50_LLama_complex_3_LSTM.obj
erqaser_movie_50_LLama_0_LSTM.obj
erqaser_movie_10_LLama_complex_2_LSTM.obj
erqaser_movie_10_LLama_3_LSTM.obj
erqaser_movie_10_LLama_complex_4_LSTM.obj
erqaser_movie_50_LLama_complex_0_LSTM.obj
eraser_movie_50_paraphrase_1_LSTM.obj
erqaser_movie_10_LLama_2_LSTM.obj
erqaser_movie_50_LLama_2_LSTM.obj
eraser_movie_10_ROS_4_LSTM.obj
erqaser_movie_10_LLama_4_LSTM.obj

In [9]:
3+5

8